In [1]:
import torch
from torchvision import models, transforms
import cv2
import numpy as np

# Load a pre-trained Faster R-CNN model
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Define a transform to normalize the images
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Function to perform object detection
def detect_animals(image_path):
    # Read the image
    img = cv2.imread(image_path)
    
    # Convert the image to RGB (as OpenCV loads in BGR)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Apply the transform
    image_tensor = transform(img_rgb).unsqueeze(0)  # Add batch dimension
    
    # Perform inference
    with torch.no_grad():
        predictions = model(image_tensor)
    
    # Extract bounding boxes, labels, and scores
    boxes = predictions[0]['boxes'].numpy()
    labels = predictions[0]['labels'].numpy()
    scores = predictions[0]['scores'].numpy()
    
    # Filter predictions by confidence threshold
    threshold = 0.5
    valid_boxes = boxes[scores >= threshold]
    valid_labels = labels[scores >= threshold]
    
    # Draw bounding boxes
    for box in valid_boxes:
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    # Display the image with bounding boxes
    cv2.imshow("Detected Animals", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Test the function with an image path
detect_animals('D:\superior\semester_4th\PAI LAB\goat.jpg')


<>:51: SyntaxWarning: invalid escape sequence '\s'
<>:51: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Junaid\AppData\Local\Temp\ipykernel_13888\3326259823.py:51: SyntaxWarning: invalid escape sequence '\s'
  detect_animals('D:\superior\semester_4th\PAI LAB\goat.jpg')
C:\Users\Junaid\AppData\Local\Temp\ipykernel_13888\3326259823.py:51: SyntaxWarning: invalid escape sequence '\s'
  detect_animals('D:\superior\semester_4th\PAI LAB\goat.jpg')


ModuleNotFoundError: No module named 'torch'

In [ ]:
import numpy as np
from scipy.spatial.distance import cdist

def group_herds(bounding_boxes, distance_threshold=50):
    """
    Groups animals into herds based on proximity using the Euclidean distance.
    """
    # Extract the center points of each bounding box
    centers = np.array([[(box[0] + box[2]) / 2, (box[1] + box[3]) / 2] for box in bounding_boxes])
    
    # Compute pairwise distances between center points
    distances = cdist(centers, centers)
    
    # Create herds based on distance threshold
    herds = []
    visited = set()
    
    for i in range(len(centers)):
        if i not in visited:
            herd = [i]
            visited.add(i)
            for j in range(i+1, len(centers)):
                if distances[i, j] < distance_threshold:
                    herd.append(j)
                    visited.add(j)
            herds.append(herd)
    
    return herds

# Example usage with detected bounding boxes
bounding_boxes = [(50, 50, 150, 150), (200, 200, 300, 300), (60, 60, 160, 160)]
herds = group_herds(bounding_boxes)

print("Detected Herds:", herds)


Detected Herds: [[0, 2], [1]]


In [10]:
import folium

# Example coordinates (you should use actual GPS coordinates from your application)
location_of_herd = [37.7749, -122.4194]  # Lat, Long for San Francisco

# Create a map centered on the location
mymap = folium.Map(location=location_of_herd, zoom_start=12)

# Add a marker for the herd location
folium.Marker(
    location_of_herd,
    popup="Herd detected!",
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(mymap)

# Save the map as an HTML file
mymap.save("herd_detection_map.html")

# Display a message if herd is detected
print("Herd detected at:", location_of_herd)


Herd detected at: [37.7749, -122.4194]
